# Sigils of The Codex

By the end of this Code Along session, you will learn to run inferences through **Google's Cloud Vision API**, retrieve the returned labels, and sort them by recyclabile categories. You will implement a **scoring system** to determine the prediction with the highest confidence rate for each image, and build a simple tool to **identify recyclable items** within images.

## Install Dependancies
We'll first need to install some dependancies. Here is a breakdown of the necessary modules:
- `google-cloud-vision`: The official Google Cloud Vision API client library for Python.

- `pillow`: A Python Imaging Library with Image Processing capabilities.

In [10]:
%pip install google-cloud-vision pillow

Note: you may need to restart the kernel to use updated packages.


## Dependancy Import and Environment Setup
We'll authenticate our Google Cloud Vision API client using the `serviceAccountKey.json` file.

In [11]:
import os, tempfile, json
from google.cloud import vision
from PIL import Image
from collections import OrderedDict

# Set Credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'serviceAccountKey.json'

## Load Data
`category_map.json` → maps categories (e.g., plastic, paper) to Google Cloud's Vision API labels.

In [12]:
with open('category_map.json', 'r') as f:
    category_map = json.load(f)

## Pre-processing (Image Optimization)
Resize and convert all images to JPEG format.

In [13]:
def optimize_image(image_path, max_size=(800, 800)):
    with Image.open(image_path) as img:
        img.thumbnail(max_size)
        jpeg_path = image_path + ".jpg"
        img.convert("RGB").save(jpeg_path, "JPEG", optimize=True, quality=85)
    return jpeg_path

## Object Localization
`ImageAnnotatorClient` performs Object Localization on the images. We'll construct a collection of annotations associated with each image.

In [14]:
def get_detected_objects(image_path):
    client = vision.ImageAnnotatorClient()
    try:
        with open(image_path, 'rb') as image_file:
            content = image_file.read()
        image = vision.Image(content=content)

        response = client.object_localization(image=image)
        objects = response.localized_object_annotations

        if response.error.message:
            raise Exception(f'Error: {response.error.message}')

        detected_objects = [obj.name for obj in objects]
        return detected_objects
    except Exception as e:
        return {'error': str(e)}

## Object Mapping
We'll map the detected objects to their corresponding categories to find the most relevant category for each image.

In [15]:
def get_recyclable_categories(detected_objects):
    matching_categories = set()
    for category, terms in category_map.items():
        if any(obj in terms for obj in detected_objects):
            matching_categories.add(category)
    return list(matching_categories) if matching_categories else None

## Handling Edge-Case (Multiple Objects Detected)
In the likely scenario that multiple objects are detected in an image, we will implement a scoring system to determine the best-fit category based on the object overlaps. In the scenario that there is still a tie between multiple categories, we will perform **Label Detection** on the image to get a more granular analysis of the detected objects. This will help us identify the most relevant category based on the highest confidence score across a wider range of generic labels.

`ImageAnnotatorClient` performs Label Detection on the images. We'll construct a collection of annotations associated with each image.

In [16]:
def granular_analysis_to_resolve_tie(file_name, image_path, tied_categories):
    client = vision.ImageAnnotatorClient()
    try:
        with open(image_path, 'rb') as image_file:
            content = image_file.read()
        image = vision.Image(content=content)

        response = client.label_detection(image=image)
        labels = [label.description for label in response.label_annotations]

        if response.error.message:
            raise Exception(f'Error: {response.error.message}')

        label_match_count = {category: 0 for category in tied_categories}
        for category in tied_categories:
            terms = category_map.get(category, [])
            label_match_count[category] = sum(1 for label in labels if label in terms)

        best_category = max(label_match_count, key=label_match_count.get)
        return best_category if label_match_count[best_category] > 0 else None
    except Exception as e:
        return {'error': str(e)}

def get_best_fitting_category(file_name, image_path, detected_objects, matched_categories):
    category_match_count = {category: 0 for category in matched_categories}
    for category in matched_categories:
        terms = category_map.get(category, [])
        category_match_count[category] = sum(1 for obj in detected_objects if obj in terms)

    best_category = max(category_match_count, key=category_match_count.get)
    highest_count = category_match_count[best_category]

    tied_categories = [cat for cat, count in category_match_count.items() if count == highest_count]

    if len(tied_categories) > 1:
        return granular_analysis_to_resolve_tie(file_name, image_path, tied_categories)

    return best_category

## Putting it to the test!

Upload an image and run an inference through our pipeline to retrieve detected objects and categories.


In [17]:
def analyze_image(image_path):
    optimized_path = optimize_image(image_path)

    detected_objects = get_detected_objects(optimized_path)
    if isinstance(detected_objects, dict) and 'error' in detected_objects:
        print(f"Error detecting objects: {detected_objects['error']}")
        return

    if not detected_objects:
        print("Category: No match")
        return

    recyclable_categories = get_recyclable_categories(detected_objects)
    if recyclable_categories:
        best_category = get_best_fitting_category(
            file_name=os.path.basename(image_path),
            image_path=image_path,
            detected_objects=detected_objects,
            matched_categories=recyclable_categories
        )
        if best_category:
            print(f"{best_category} is recyclable!")
            return

    print("Category: No match")

test_image = "sample.jpg"
analyze_image(test_image)

Paper is recyclable!
